In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
# Replace 'your_file.csv' with the path to your actual dataset
data = pd.read_csv('pat_dataset.csv')

# Define input (X) and output (y) columns
input_columns = ['dimx', 'dimy', 'dimz', 'buffer_size', 'packet_size_min', 'packet_size_max',
                 'routing_type', 'selection_strategy', 'traffic_type', 'injection_rate']
output_columns = ['steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power',
                  'avg_power_per_router', 'layer_area', 'area_per_core']

X = data[input_columns]
y = data[output_columns]

# Preprocessing pipeline
# OneHotEncode categorical columns and Standardize numerical columns
categorical_cols = ['routing_type', 'selection_strategy', 'traffic_type']
numerical_cols = [col for col in input_columns if col not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Create the pipeline with preprocessing and model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='variance_weighted')

# Display results
print("Mean Squared Error for each output:", mse)
print("Weighted R2 Score:", r2)


Mean Squared Error for each output: [1.69970462e+01 1.62926494e+01 7.83984046e-02 4.69594195e-02
 9.46624266e-03 5.45474934e-03 7.23712576e-04 6.25039666e-04
 1.17421161e+08 5.86692293e-16 1.67889097e-16 1.28354052e-16
 3.16472000e-21 2.93687462e+07 0.00000000e+00]
Weighted R2 Score: 0.9999999978381127


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Embedding, Flatten

# Load dataset
data = pd.read_csv('pat_dataset.csv')

# Separate inputs and outputs
input_columns = ['dimx', 'dimy', 'dimz', 'buffer_size', 'packet_size_min',
                 'packet_size_max', 'routing_type', 'selection_strategy',
                 'traffic_type', 'injection_rate']
output_columns = ['steady_state_temp_L0', 'steady_state_temp_L1',
                  'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1',
                  'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power',
                  'avg_routers_power', 'avg_power_per_router',
                  'layer_area', 'area_per_core']

X = data[input_columns]
y = data[output_columns]

# One-hot encode categorical features
categorical_features = ['routing_type', 'selection_strategy', 'traffic_type']
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define neural network model
input_shape = X_train.shape[1]
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1])  # Output layer with number of outputs equal to the target variables
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)

# Evaluate model
results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (MSE): {results[0]}")
print(f"Test Mean Absolute Error: {results[1]}")

# Make predictions
y_pred = model.predict(X_test)

# Calculate individual metrics for each output variable
mse_per_output = np.mean((y_test - y_pred) ** 2, axis=0)
mae_per_output = np.mean(np.abs(y_test - y_pred), axis=0)

# Display results for each output
print("Performance Metrics for Each Output:")
for i, col in enumerate(output_columns):
    print(f"Output: {col}")
    print(f" - MSE: {mse_per_output[i]:.4f}")
    print(f" - MAE: {mae_per_output[i]:.4f}")
    print("")

# Overall performance metrics
overall_mse = np.mean(mse_per_output)
overall_mae = np.mean(mae_per_output)

print("Overall Metrics:")
print(f" - Mean MSE across outputs: {overall_mse:.4f}")
print(f" - Mean MAE across outputs: {overall_mae:.4f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


520/520 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 12801769574760448.0000 - mae: 32131790.0000 - val_loss: 13373534948556800.0000 - val_mae: 32893030.0000
Epoch 2/100
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13203551954141184.0000 - mae: 32655716.0000 - val_loss: 13372308735393792.0000 - val_mae: 32890766.0000
Epoch 3/100
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13047841136050176.0000 - mae: 32443174.0000 - val_loss: 13369505195491328.0000 - val_mae: 32885602.0000
Epoch 4/100
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 12803391998656512.0000 - mae: 32098520.0000 - val_loss: 13364890253131776.0000 - val_mae: 32877002.0000
Epoch 5/100
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13028339837042688.0000 - mae: 32319528.0000 - val_loss: 13358051591454720.0000 - val_mae: 32864250.0000
Epoch 6/100
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13643246009843712.0000 - mae: 33187998.0000 - val_loss: 13347869297737728.0000 - val_mae: 32844696.0000
Epoch 7/100
520/520 ━━

<ipython-input-4-c663460f4432>:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f" - MSE: {mse_per_output[i]:.4f}")
<ipython-input-4-c663460f4432>:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f" - MAE: {mae_per_output[i]:.4f}")


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load your dataset
df = pd.read_csv('pat_dataset.csv')  # Replace with your actual dataset path

# Display the first few rows of the dataset (optional, to inspect data)
print(df.head())

# Identify non-numeric columns (categorical columns)
categorical_columns = df.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)

# Convert categorical columns to numerical using one-hot encoding
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Define the target columns (output variables)
target_columns = [
    'steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0',
    'router_avg_temp_L1', 'core_avg_temp_L0', 'core_avg_temp_L1',
    'mem_avg_temp_L0', 'mem_avg_temp_L1', 'total_area', 'avg_power',
    'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
    'layer_area', 'area_per_core'
]

# Initialize dictionary to store the results
results = {}

# Split dataset into X (features) and y (target variables)
X = df.drop(columns=target_columns)  # Drop target columns to get features

# Iterate over each target and perform regression
for target in target_columns:
    # Extract the current target variable
    y = df[target]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a Linear Regression model (you can replace this with other regression models)
    model = LinearRegression()  # Example model, replace with other models as needed
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store the metrics for this target variable
    results[target] = {
        'MSE': mse,
        'MAE': mae,
        'R²': r2
    }

# Output the results for each target variable
for target, metrics in results.items():
    print(f"Results for {target}:")
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R²: {metrics['R²']}")
    print("-" * 40)


   dimx  dimy  dimz  buffer_size  packet_size_min  packet_size_max  \
0     3     2     2            4                4                8   
1     3     2     2            4                4                8   
2     3     2     2            4                4                8   
3     3     2     2            4                4                8   
4     3     2     2            4                4                8   

  routing_type selection_strategy traffic_type  injection_rate  ...  \
0          xyz            thermal       random            0.01  ...   
1          xyz            thermal       random            0.02  ...   
2          xyz            thermal       random            0.03  ...   
3          xyz            thermal       random            0.04  ...   
4          xyz            thermal       random            0.05  ...   

   core_avg_temp_L1  mem_avg_temp_L0  mem_avg_temp_L1  total_area  \
0         25.888400        25.666300        25.595383  28332600.0   
1         26.0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load your dataset
df = pd.read_csv('pat_dataset.csv')

# Columns containing the target variables
target_columns = ['steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
                  'layer_area', 'area_per_core']

# Split the dataset into features (X) and target variables (y)
X = df.drop(columns=target_columns)  # Features
y = df[target_columns]  # Targets

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler_X = StandardScaler()
scaler_y = {}

# Scale the feature data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scale the target variables (y)
for target in target_columns:
    scaler_y[target] = StandardScaler()
    y_train[target] = scaler_y[target].fit_transform(y_train[[target]])
    y_test[target] = scaler_y[target].transform(y_test[[target]])

# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, target):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print the results
    print(f"Results for {target}:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print("-" * 40)

# Train and evaluate models for each target variable
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Regressor': SVR()
}

# Evaluate each model for each target variable
for target in target_columns:
    print(f"\nEvaluating models for {target}...")

    for model_name, model in models.items():
        print(f"\nModel: {model_name}")
        evaluate_model(model, X_train_scaled, X_test_scaled, y_train[target], y_test[target], target)

# Optionally, you can also apply other models and optimizations



Evaluating models for steady_state_temp_L0...

Model: Linear Regression
Results for steady_state_temp_L0:
MSE: 0.49606552122682745
MAE: 0.5315647874219586
R²: 0.5100783891806185
----------------------------------------

Model: Decision Tree
Results for steady_state_temp_L0:
MSE: 0.005983285730782671
MAE: 0.020125885275033985
R²: 0.9940908189386594
----------------------------------------

Model: Random Forest
Results for steady_state_temp_L0:
MSE: 0.003232749677317736
MAE: 0.019136184493298625
R²: 0.9968072888327928
----------------------------------------

Model: Support Vector Regressor
Results for steady_state_temp_L0:
MSE: 0.07287939272515927
MAE: 0.13586405493715317
R²: 0.9280232389641891
----------------------------------------

Evaluating models for steady_state_temp_L1...

Model: Linear Regression
Results for steady_state_temp_L1:
MSE: 0.49927961589653364
MAE: 0.5330637775287035
R²: 0.5062256306810851
----------------------------------------

Model: Decision Tree
Results for s

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load your dataset
df = pd.read_csv('pat_dataset.csv')

# Columns containing the target variables
target_columns = ['steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
                  'layer_area', 'area_per_core']

# Split the dataset into features (X) and target variables (y)
X = df.drop(columns=target_columns)  # Features
y = df[target_columns]  # Targets

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler_X = StandardScaler()
scaler_y = {}

# Scale the feature data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scale the target variables (y)
for target in target_columns:
    scaler_y[target] = StandardScaler()
    y_train[target] = scaler_y[target].fit_transform(y_train[[target]])
    y_test[target] = scaler_y[target].transform(y_test[[target]])

from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize the additional models
models=({
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'ElasticNet': ElasticNet()
})

# Function to evaluate models (same as before)
def evaluate_model(model, X_train, X_test, y_train, y_test, target):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print the results
    print(f"Results for {target}:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print("-" * 40)

# Evaluate each model for each target variable
for target in target_columns:
    print(f"\nEvaluating models for {target}...")

    for model_name, model in models.items():
        print(f"\nModel: {model_name}")
        evaluate_model(model, X_train_scaled, X_test_scaled, y_train[target], y_test[target], target)



Evaluating models for steady_state_temp_L0...

Model: Gradient Boosting
Results for steady_state_temp_L0:
MSE: 0.0672082395482736
MAE: 0.18281762749140512
R²: 0.933624153320728
----------------------------------------

Model: AdaBoost
Results for steady_state_temp_L0:
MSE: 0.27145204679340235
MAE: 0.4464060113977227
R²: 0.7319099628284151
----------------------------------------

Model: ElasticNet
Results for steady_state_temp_L0:
MSE: 0.9596050294203804
MAE: 0.780451375998598
R²: 0.052279947613928734
----------------------------------------

Evaluating models for steady_state_temp_L1...

Model: Gradient Boosting
Results for steady_state_temp_L1:
MSE: 0.06420044183395518
MAE: 0.17706143601911867
R²: 0.9365074566089912
----------------------------------------

Model: AdaBoost
Results for steady_state_temp_L1:
MSE: 0.24618600395856477
MAE: 0.4215740851082759
R²: 0.7565285363763472
----------------------------------------

Model: ElasticNet
Results for steady_state_temp_L1:
MSE: 0.954828

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Load your dataset
df = pd.read_csv('pat_dataset.csv')

# Columns containing the target variables
target_columns = ['steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
                  'layer_area', 'area_per_core']

# Split the dataset into features (X) and target variables (y)
X = df.drop(columns=target_columns)  # Features
y = df[target_columns]  # Targets

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler_X = StandardScaler()
scaler_y = {}

# Scale the feature data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scale the target variables (y)
for target in target_columns:
    scaler_y[target] = StandardScaler()
    y_train[target] = scaler_y[target].fit_transform(y_train[[target]])
    y_test[target] = scaler_y[target].transform(y_test[[target]])

# Reshape the feature data for CNN (samples, timesteps, features)
X_train_scaled_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Build CNN model for regression
cnn_model = Sequential()

# 1D Convolutional Layer
cnn_model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled_reshaped.shape[1], 1)))

# Max Pooling Layer with pool_size=1 to prevent reducing dimensions to 0
cnn_model.add(MaxPooling1D(pool_size=1))

# 2nd Convolutional Layer
cnn_model.add(Conv1D(128, kernel_size=3, activation='relu'))

# Max Pooling Layer with pool_size=1 to maintain the dimension
cnn_model.add(MaxPooling1D(pool_size=1))

# Flatten the output from the convolutional layers
cnn_model.add(Flatten())

# Fully Connected Layer
cnn_model.add(Dense(64, activation='relu'))

# Output layer for regression (one neuron per target)
cnn_model.add(Dense(len(target_columns)))

# Compile the CNN model
cnn_model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the CNN model
cnn_history = cnn_model.fit(X_train_scaled_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled_reshaped, y_test))

# Predict with the CNN model
y_pred_cnn = cnn_model.predict(X_test_scaled_reshaped)

# Inverse transform the predictions and actual values to original scale
y_pred_cnn = pd.DataFrame(y_pred_cnn, columns=target_columns)
y_test_inv = y_test.copy()

for target in target_columns:
    y_pred_cnn[target] = scaler_y[target].inverse_transform(y_pred_cnn[[target]])
    y_test_inv[target] = scaler_y[target].inverse_transform(y_test_inv[[target]])

# Calculate and print evaluation metrics for CNN
for target in target_columns:
    mse = mean_squared_error(y_test_inv[target], y_pred_cnn[target])
    mae = mean_absolute_error(y_test_inv[target], y_pred_cnn[target])
    r2 = r2_score(y_test_inv[target], y_pred_cnn[target])

    print(f"Results for CNN Regression ({target}):")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print("-" * 40)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


520/520 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.4262 - val_loss: 0.1117
Epoch 2/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0971 - val_loss: 0.0823
Epoch 3/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0726 - val_loss: 0.0517
Epoch 4/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0491 - val_loss: 0.0464
Epoch 5/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0418 - val_loss: 0.0349
Epoch 6/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.0369 - val_loss: 0.0319
Epoch 7/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0311 - val_loss: 0.0250
Epoch 8/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0238 - val_loss: 0.0261
Epoch 9/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0207 - val_loss: 0.0224
Epoch 10/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0185 - val_loss: 0.0227
Epoch 11/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0188 - val_loss: 0.0187
Epoch 12/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models

# Load your dataset
df = pd.read_csv('pat_dataset.csv')

# Columns containing the target variables
target_columns = ['steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
                  'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
                  'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
                  'layer_area', 'area_per_core']

# Split the dataset into features (X) and target variables (y)
X = df.drop(columns=target_columns)  # Features
y = df[target_columns]  # Targets

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Apply log transformation to target variables (for targets with large range of values)
y_log = np.log1p(y)  # Apply log1p transformation to all target columns

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Initialize RobustScaler for features
scaler_X = RobustScaler()
scaler_X.fit(X_train)
X_train_scaled = scaler_X.transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Initialize StandardScaler for target variables (log-transformed)
scaler_y = {}
for target in target_columns:
    scaler_y[target] = StandardScaler()
    y_train[target] = scaler_y[target].fit_transform(y_train[[target]])
    y_test[target] = scaler_y[target].transform(y_test[[target]])

# CNN Model (without MaxPooling1D)
def build_cnn_model(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# CNN Regression for each target variable
for target in target_columns:
    print(f"\nEvaluating CNN model for {target}...")

    # Reshape X data for CNN
    X_train_cnn = np.expand_dims(X_train_scaled, axis=-1)
    X_test_cnn = np.expand_dims(X_test_scaled, axis=-1)

    # Build and train CNN model
    cnn_model = build_cnn_model(input_shape=(X_train_cnn.shape[1], 1))
    cnn_model.fit(X_train_cnn, y_train[target], epochs=50, batch_size=32, verbose=0)

    # Predict with CNN model
    y_pred_cnn = cnn_model.predict(X_test_cnn)

    # Inverse transform the predictions and targets (without reshape)
    y_pred_cnn_rescaled = scaler_y[target].inverse_transform(y_pred_cnn)
    y_test_rescaled = scaler_y[target].inverse_transform(y_test[target].values.reshape(-1, 1))

    # Calculate evaluation metrics for CNN model
    mse_cnn = mean_squared_error(y_test_rescaled, y_pred_cnn_rescaled)
    mae_cnn = mean_absolute_error(y_test_rescaled, y_pred_cnn_rescaled)
    r2_cnn = r2_score(y_test_rescaled, y_pred_cnn_rescaled)

    # Print CNN results
    print(f"MSE: {mse_cnn}")
    print(f"MAE: {mae_cnn}")
    print(f"R²: {r2_cnn}")
    print("-" * 40)




Evaluating CNN model for steady_state_temp_L0...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
MSE: 0.00043396271717479083
MAE: 0.012335052719150315
R²: 0.9925083713581937
----------------------------------------

Evaluating CNN model for steady_state_temp_L1...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE: 0.00025787358101189435
MAE: 0.009295801393801118
R²: 0.9956007427609966
----------------------------------------

Evaluating CNN model for router_avg_temp_L0...


...

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE: 7.263741311915782e-23
MAE: 3.91178182853345e-12
R²: 0.9960327477784054
----------------------------------------

Evaluating CNN model for layer_area...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
MSE: 7.294416493790209e-06
MAE: 0.0021510899674625156
R²: 0.9999891797478107
----------------------------------------

Evaluating CNN model for area_per_core...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE: 8.69138002189672e-14
MAE: 1.8897150256091234e-07
R²: -3060453639324726.5
----------------------------------------
